<a href="https://colab.research.google.com/github/tonchan1216/bitbot/blob/master/notebook/random_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random
import sys
from google.colab import drive

from datetime import datetime
from matplotlib import pyplot as plt
from tensorflow.keras.utils import Progbar

lib_path = '/content/drive/My Drive/Colab Notebooks/bitbot/scripts'
if (lib_path not in sys.path):
  sys.path.append(lib_path)

In [ ]:
mode = 'test'
name = 'random'

mound_dir = '/content/drive/'
drive.mount(mound_dir)

root_dir = 'My Drive/Colab Notebooks/bitbot/'
mdl_dir = mound_dir + root_dir + 'models'
nov_path = mound_dir + root_dir + f'training_data/btc_{mode}.csv'
csv_path = mound_dir + root_dir + f'result_data/{name}_{mode}.csv'

df = pd.read_csv(nov_path)
df['Date'] = pd.to_datetime(df['Date'], format = '%Y-%m-%d')
df.head()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


,Date,BTC
0,2017-01-02,120750
1,2017-01-03,122408
2,2017-01-04,123138
3,2017-01-05,143743
4,2017-01-06,118936


In [ ]:
class Main:
    def __init__(self, env, mdl_dir, name, episodes_times = 1000, mode = 'test'):
        self.env = env
        self.mdl_dir = mdl_dir
        self.episodes_times = episodes_times
        self.mode = mode
        self.name = name

        with open(csv_path, 'w') as f:
            row = 'FixedProfit,TradeTimes,TradeWin'
            print(row, file=f)


    def play_game(self):

        for episode in range(self.episodes_times):

            if (episode % 10 == 0):
                metrics_names = ['FixedProfit','TradeTimes','TradeWin']
                pb_i = Progbar(10, stateful_metrics=metrics_names)
                p_mean,trade_time,win_time = np.array([]),np.array([]),np.array([])

            state = self.env.reset()
            done = False
        
            while not done:
                action = random.randrange(3)
                next_state, reward, done, info = self.env.step(action)
                state = next_state

            p_mean,trade_time,win_time = np.append(p_mean,info['cur_revenue']),np.append(trade_time,info['trade_time']),np.append(win_time,info['trade_win'])
            values=[('FixedProfit',int(np.mean(p_mean))), ('TradeTimes',int(np.mean(trade_time))), ('TradeWin',int(np.mean(win_time)))]
            pb_i.add(1, values=values)
            with open(csv_path, 'a') as f:
                row = str(info['cur_revenue']) + ',' + str(info['trade_time']) + ',' + str(info['trade_win'])
                print(row, file=f)

In [ ]:
import environment
initial_money=100000
episodes_times = 100
commission = 0

env = environment.Environment(df, initial_money = initial_money, mode = mode, commission = commission)
main = Main(env, mdl_dir, name, episodes_times, mode)
main.play_game()

10/10 [==============================] - 1s 54ms/step - FixedProfit: 803745.0000 - TradeTimes: 304.0000 - TradeWin: 167.0000
